In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import scipy
import numpy as np
import pandas as pd
import os
os.chdir('/home/cat/code/manifolds')

from calcium import calcium
from wheel import wheel
from visualize import visualize
from tqdm import trange


import numpy as np
import scipy
from scipy.io import loadmat

from tqdm import tqdm, trange

Autosaving every 180 seconds


In [2]:
######################################################
######################################################
######################################################

# data = loadmat('/media/cat/4TBSSD/donato/HEX5_K5_medium__T1800s_dT500ms_cT1s_sKInf_eF10.0mHz_eM6_nS0/HEX5_K5_medium__T1800s_dT500ms_cT1s_sKInf_eF10.0mHz_eM6_nS0_ACTIVITY-RASTER_sums.mat')
# print (len(data))

# print (data.keys())
# print (data['activity_raster'].shape)
# print (data['activity_raster_threshold'].shape)
# print (data['activity_raster_peaks'].shape)

# ######################################################
# ######################################################
# ######################################################
# peaks = data['activity_raster_peaks']
# #print (peaks)
# rasters = data['activity_raster']
# print ("rasters: ", rasters.shape)
# sums = rasters.sum(axis=1)
# sample_rate = 2.
# t=np.arange(sums.shape[0])/sample_rate

# ################################################
# fig=plt.figure()
# #
# plt.plot(t, sums, c='black', label='mua')

# #
# sums_peaks = sums.copy()
# sums_peaks = sums_peaks*np.nan
# sums_peaks[peaks-1]= sums[peaks-1]

# plt.scatter(t, 
#             sums_peaks,
#         c='red', label = 'peaks for ensemble algorithm')
# plt.plot(t, rasters[:,3]*20,c='blue', label='example bin neuron')
# plt.legend()
# plt.show()

In [3]:
##########################################################
############ ALTERNATIVE: GET RASTERS FROM DF_F ##########
##########################################################

#rasters = np.load('/media/cat/4TBSSD/donato/HEX5_K5_medium__T1800s_dT500ms_cT1s_sKInf_eF10.0mHz_eM6_nS0/dF_F_binned_sum.npy')
#rasters = np.load('/media/cat/4TBSSD/donato/HEX5_K5_medium__T1800s_dT500ms_cT1s_sKInf_eF10.0mHz_eM6_nS0/dF_F_binned_max.npy')
#rasters = np.load('/media/cat/4TBSSD/donato/HEX5_K5_medium__T1800s_dT500ms_cT1s_sKInf_eF10.0mHz_eM6_nS0/dF_F_binned_mean.npy')
rasters = np.load('/media/cat/4TBSSD/donato/upphase/F_upphase_binned_sum.npy')
print (rasters.shape)
print (rasters)

plt.imshow(rasters.T,
          aspect='auto',
          #vmin=0,
          #vmax=1
           interpolation='none'
          )
plt.show()

(3716, 531)
[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  5.  0.]
 [ 0.  0.  0. ...  0.  7.  0.]
 [ 0.  0.  0. ...  0. 11.  0.]]


In [4]:
#######################################################
################ Z-SCORE RASTERS ######################
#######################################################
Zib = np.zeros(rasters.shape)
print ("Zib: ", Zib.shape)

# compute standard deviations
sigma = np.std(rasters,axis=0)
print ("sigma: ", sigma.shape)

# compute standard deviations
means = np.mean(rasters,axis=0)
print ("means: ", means.shape)


# 
for t in trange(Zib.shape[0]):
    for c in range(Zib.shape[1]):
        Zib[t,c]= (rasters[t,c]-means[c])/sigma[c]
    



  4%|▍         | 143/3716 [00:00<00:02, 1429.33it/s]

Zib:  (3716, 531)
sigma:  (531,)
means:  (531,)


100%|██████████| 3716/3716 [00:02<00:00, 1451.45it/s]


In [5]:
fig=plt.figure()
ax=plt.subplot(2,1,1)
plt.imshow(rasters.T,
          aspect='auto',
          extent=[0,rasters.shape[0]/2,0,rasters.shape[1]])
plt.xticks([])
plt.ylabel("Neuron ID")
plt.title("Raw rasters (sumed + binned to 2Hz)")
ax=plt.subplot(2,1,2)
print (np.min(Zib), np.max(Zib))
plt.title("Z scored rasters ")
plt.imshow(Zib.T,
           vmin=np.min(Zib),
           vmax=8,
           aspect='auto',
           extent=[0,rasters.shape[0]/2,0,rasters.shape[1]])
plt.ylabel("Neuron ID")
plt.xlabel("Time (sec)")
plt.show()

fig = plt.figure()

y = np.histogram(Zib.ravel(), bins=np.arange(1,25,1))
plt.title("distribution of z-scored values ")
plt.plot(y[1][:-1],
        y[0])
plt.semilogy()
plt.show()

-0.2924512028694153 59.327293395996094


In [7]:
#######################################################
################## LOAD ASSEMBLIES ####################
#######################################################
if True:
#    data = loadmat('/media/cat/4TBSSD/donato/HEX5_K5_medium__T1800s_dT500ms_cT1s_sKInf_eF10.0mHz_eM6_nS0/HEX5_K5_medium__T1800s_dT500ms_cT1s_sKInf_eF10.0mHz_eM6_nS0_ICA-ASSEMBLIES.mat')
    data = loadmat('/media/cat/4TBSSD/donato/upphase/F_upphase_binarized_sum.mat')
    print (data.keys())
    ens_mp = data['mp_assemblies'].squeeze()-1
    ens_cs = data['cs_assemblies'].squeeze()-1

    # 
    print ("Cs: ", ens_cs.shape)
    print ("mp: ", ens_mp.shape)
else:
    data = loadmat('/media/cat/4TBSSD/donato/upphase/Upphase_SGC-ASSEMBLIES.mat')
    print (data.keys())
    assemblies = data['assemblies'].squeeze()-1
    assembly_pattern_detection = data['assembly_pattern_detection'].squeeze()
    
    print ("assembiles ", assemblies.shape)
    print ("assembly_pattern_detection: ", assembly_pattern_detection.shape)
    
    
#
w = np.zeros(rasters.shape[1])


dict_keys(['__header__', '__version__', '__globals__', 'cs_assembly_vectors', 'cs_assemblies', 'ks_alpha', 'mp_assembly_vectors', 'mp_assemblies'])
Cs:  (25,)
mp:  (63,)


In [18]:
######################################################
######### MAKE OUTER PRODUCT AND PROJECTION ##########
######################################################
#selected_ensemble = assemblies.copy()
selected_ensemble = ens_cs.copy()
ensemble_ids = np.arange(selected_ensemble.shape[0])


#
times=np.arange(rasters.shape[0])/2.
#
ctr=0
fig=plt.figure()
ensemble_times = []
ensemble_stacks = []

# 
thresh = 0
for ensemble_id in ensemble_ids:
    w = np.zeros(rasters.shape[1])
    idx = selected_ensemble[ensemble_id].squeeze()
    #print (idx)
    w[idx] = 1

    W = np.outer(w,w)
    
    #
    f = np.zeros(Zib.shape[0])
    s = np.zeros(Zib.shape)
    for t in trange(Zib.shape[0]):
        temp1 = np.matmul(Zib[t].T,W)
        temp2 = np.matmul(temp1,Zib[t])
        f[t]=temp2
        
        #
        for i in idx:
            if Zib[t,i]>=thresh:
                s[t,i]=1
            
    #
    ensemble_times.append(f)
    ensemble_stacks.append(s)
    
    # 
    #times = np.log(f+1E-5)
    if True:
        plt.plot(times,f+ctr*10000)
    else:
        plt.plot(times, np.log(f+0.0001)+ctr*25)
    plt.xlabel("Time (sec)")
    plt.ylabel("Ensemble ID")
    
    ctr+=1

#
ticks1 = np.arange(0, 10000*ctr, 10000)
ticks2 = np.arange(len(ensemble_ids))
    
ensemble_times = np.stack(ensemble_times)
plt.yticks(ticks1, ticks2)
plt.show()


100%|██████████| 3716/3716 [00:00<00:00, 16181.08it/s]


In [77]:
######################################################
################## LOOK AT EACH ######################
######################################################
id_ = 0
sample_rate=2

######################
fig=plt.figure()
ax=plt.subplot(3,1,1)
e = ensemble_times[id_]
plt.plot(times,e)

# 
plt.xlim(0,e.shape[0]/sample_rate)
plt.title("Ensemble: "+str(id_))
plt.ylabel("Ensemble presence (projection)")
plt.ylim(0,10000)


##################
ax=plt.subplot(3,1,2)
ee = ensemble_stacks[id_]
plt.imshow(ee.T,
          aspect='auto',
          interpolation='none',
          extent=[0,ee.shape[0]/sample_rate, 0,ee.shape[1]])
plt.xlabel("Time (sec)")
plt.ylabel("Neuron ID")

##################
ax=plt.subplot(3,1,3)
ee = ensemble_stacks[id_]
sums2 = ee.sum(axis=1)
idx = np.where(sums2>0)[0]
print (idx)
temp = rasters[idx]
print (temp.shape)
plt.imshow(temp,
          aspect='auto',
          interpolation='none',
          extent=[0,temp.shape[0]/sample_rate, 
                  0,temp.shape[1]]
          )
plt.xlabel("Time - discontinuous (sec)")
plt.ylabel("Removed zero-activity frames\nNeuron ID")

plt.show()


[   0    1    2 ... 3713 3714 3715]
(2349, 531)


In [93]:
##################################################
################ LOAD FOOTPRINTS #################
##################################################
root_dir = '/media/cat/4TBSSD/donato/binarized_traces/'
animal_id = ''
session = ''

#
c = calcium.Calcium()
c.animal_id = animal_id
c.root_dir = root_dir
c.session = session
c.recompute = False
c.load_suite2p()

#
c.load_binarization()

# 
c.load_footprints()




(531, 55740)


In [95]:
######################################################
################ VISUALIZE ENSEMBLES #################
######################################################
selected_ensemble = ens_cs.copy()
ensemble_ids = np.arange(selected_ensemble.shape[0])

ensemble_ids = np.arange(8)
cmap = plt.get_cmap("viridis", Zib.shape[1])

fig=plt.figure()

ctr2=0
for ensemble_id in ensemble_ids:
    ax=plt.subplot(3,3,ctr2+1)
    print ("ensemble stack: ", ensemble_times.shape) 
    ens_times = ensemble_times[ensemble_id]
    #print (ens_times.shape)

    idx = np.where(ens_times>50)[0]
    print ("# of occurances: of ensemble over threshold", idx.shape)

    # 
    norms = np.zeros(Zib.shape[1])
    print ("norms: ", norms.shape, "Zib: ", Zib.shape)

    # 
    for ctr,i in enumerate(idx):
        activity = Zib[i]
        idx2 = np.where(activity>0)[0]
        norms[idx2]+=1

    norms = (norms-np.min(norms))/(np.max(norms)-np.min(norms))

    # 
    for n in range(Zib.shape[1]):

        plt.plot(c.contours[n][:,0],
                 c.contours[n][:,1],
                 #color=cmap(n),
                 c='black',
                 linewidth=3,
                 alpha=norms[n]
                )
    plt.xlim(0,512)
    plt.ylim(0,512)
    ctr2+=1
    
plt.show()


ensemble stack:  (8, 3716)
# of occurances: of ensemble over threshold (3366,)
norms:  (531,) Zib:  (3716, 531)
ensemble stack:  (8, 3716)
# of occurances: of ensemble over threshold (2908,)
norms:  (531,) Zib:  (3716, 531)
ensemble stack:  (8, 3716)
# of occurances: of ensemble over threshold (3128,)
norms:  (531,) Zib:  (3716, 531)
ensemble stack:  (8, 3716)
# of occurances: of ensemble over threshold (3176,)
norms:  (531,) Zib:  (3716, 531)
ensemble stack:  (8, 3716)
# of occurances: of ensemble over threshold (3225,)
norms:  (531,) Zib:  (3716, 531)
ensemble stack:  (8, 3716)
# of occurances: of ensemble over threshold (2563,)
norms:  (531,) Zib:  (3716, 531)
ensemble stack:  (8, 3716)
# of occurances: of ensemble over threshold (3130,)
norms:  (531,) Zib:  (3716, 531)
ensemble stack:  (8, 3716)
# of occurances: of ensemble over threshold (3253,)
norms:  (531,) Zib:  (3716, 531)


In [48]:
######################################################
############# VISUALIZE ENSEMBLES OVER TIME ##########
######################################################
selected_ensemble = ens_cs.copy()
ensemble_ids = np.arange(selected_ensemble.shape[0])

ensemble_id = 1
print ("ensemble stack: ", ensemble_times.shape) 
ens_times = ensemble_times[ensemble_id]
print (ens_times.shape)

idx = np.where(ens_times>1000)[0]
print (idx.shape)

cmap = plt.get_cmap("viridis", 531)

for ctr,i in enumerate(idx):
    ax=plt.subplot(5,8,ctr+1)
    activity = c.F_upphase_bin[:,i]
    
    idx2 = np.where(activity>0)[0]
    #print ("idx2: ", idx2)
    for n in idx2:
        plt.plot(c.contours[n][:,0],
                 c.contours[n][:,1],
                 color=cmap(n),
                )
    plt.xlim(0,512)
    plt.ylim(0,512)
    plt.title(str(i))
plt.show()

        

ensemble stack:  (25, 3716)
(3716,)
(86,)


ValueError: num must be 1 <= num <= 40, not 41